### Preprocessing von Bruno

In [3]:
import numpy as np
import pandas as pd
from os import path
import utils.StreamLolGame as s

train_labels = "train_augmented.csv"
test_labels = "test.csv"
data_dir = "processed"

def __process_labels__(raw_y_data):
    """Takes raw labels and seperates them into the correct 'heads'

    :param raw_y_data: Raw inputd ata
    :return: A tuple of arrays, where each array contains all lables for a given output
    """
    valence = np.array([raw_y_data['V_Neg'], raw_y_data['V_Neut'], raw_y_data['V_Pos']])
    arousal = np.array([raw_y_data['A_Neut'], raw_y_data['A_Pos']])

    laning = raw_y_data['Laning']
    shopping = raw_y_data['Shopping']
    returning = raw_y_data['Returning']
    roaming = raw_y_data['Roaming']
    fighting = raw_y_data['Fighting']
    pushing = raw_y_data['Pushing']
    defending = raw_y_data['Defending']
    dead = raw_y_data['Dead']

    game_events = np.array([laning, shopping, returning, roaming, fighting, pushing, defending, dead])
    return valence, arousal, game_events

In [2]:
y_train_data = pd.read_csv(train_labels)

print("train", y_train_data.to_numpy().shape)
count = 0
for i in range(len(y_train_data)):
    if(path.exists("./processed/" + y_train_data["File"][i]) == False):
        count += 1
        y_train_data = y_train_data.drop(i)
print(count)
print("train", y_train_data.to_numpy().shape)
y_train_data.to_csv("train_clean.csv")

y_test_data = pd.read_csv(test_labels)

print("test", y_test_data.to_numpy().shape)
count = 0
for i in range(len(y_test_data)):
    if(path.exists("./processed/" + y_test_data["File"][i]) == False):
        count += 1
        y_test_data = y_test_data.drop(i)
print(count)
print("test", y_test_data.to_numpy().shape)
y_test_data.to_csv("test_clean.csv")

train (11030, 16)
4297
train (6733, 16)
test (1379, 15)
542
test (837, 15)


In [5]:
%%time

face_data = []
game_data = []
valence_labels = []
arousal_labels = []
game_event_labels = []

face_game_data = pd.read_csv("train_clean.csv")

shape = face_game_data.shape[0]

inputs = np.empty((0,128,128,3))
valence_labels = np.empty((0,3))
arousal_labels = np.empty((0,2))
game_event_labels = np.empty((0,8))

print(shape)

for i in range(0, shape):
    raw_y_data = face_game_data.iloc[i]
    proc_y_data = __process_labels__(raw_y_data)
    
    valence = np.array([proc_y_data[0]] * 5) 
    valence_labels = np.vstack((valence_labels, valence))
    
    arousal = np.array([proc_y_data[1]] * 5) 
    arousal_labels = np.vstack((arousal_labels, arousal))
    
    game_event = np.array([proc_y_data[2]] * 5) 
    game_event_labels = np.vstack((game_event_labels, game_event))
    
    video_file = face_game_data.iloc[i]['File']
    if path.exists("%s/%s/face.npy" % (data_dir, video_file)):
        inputs = np.vstack((inputs, np.load("%s/%s/game.npy" % (data_dir, video_file))[(0,6,13,19), :, :, :]))
    else:
        raise ValueError('File with index ', i, ' not found') 
    
    if(i%100 == 0):
        print(f"{i}th iteration")
        
print()

print("file", inputs.shape)
np.savez_compressed("data_game.npz", data=inputs, valence_labels=np.array(valence_labels), arousal_labels=np.array(arousal_labels), game_event_labels=np.array(game_event_labels))

6733
0th iteration
100th iteration
200th iteration
300th iteration
400th iteration
500th iteration
600th iteration
700th iteration
800th iteration
900th iteration
1000th iteration
1100th iteration
1200th iteration
1300th iteration
1400th iteration
1500th iteration
1600th iteration
1700th iteration
1800th iteration
1900th iteration
2000th iteration
2100th iteration
2200th iteration
2300th iteration
2400th iteration
2500th iteration
2600th iteration
2700th iteration
2800th iteration
2900th iteration
3000th iteration
3100th iteration
3200th iteration
3300th iteration
3400th iteration
3500th iteration
3600th iteration
3700th iteration
3800th iteration
3900th iteration
4000th iteration
4100th iteration
4200th iteration
4300th iteration
4400th iteration
4500th iteration
4600th iteration
4700th iteration
4800th iteration
4900th iteration
5000th iteration
5100th iteration
5200th iteration
5300th iteration
5400th iteration
5500th iteration
5600th iteration
5700th iteration
5800th iteration
5900

data shape:  (33665, 64, 64, 3) valence_labels shape:  (33665, 3) arousal_labels shape:  (33665, 2) game_event_labels shape:  (33665, 8)